In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("ML").getOrCreate()

24/08/03 17:07:13 WARN Utils: Your hostname, user-Lenovo-IdeaPad-S145-15API resolves to a loopback address: 127.0.1.1; using 192.168.100.5 instead (on interface wlp2s0)
24/08/03 17:07:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/03 17:07:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/03 17:07:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/03 17:07:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


24/08/03 17:07:27 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
spark

In [4]:
training = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [5]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [10]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [11]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['age', 'Experience'],outputCol='Independent Feature')

In [12]:
output = featureassembler.transform(training)

In [14]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [15]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Feature']

In [16]:
df_final = output.select('Independent Feature', 'Salary')

In [17]:
df_final.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [26]:
train_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
|         [24.0,3.0]| 20000|
|         [29.0,4.0]| 20000|
|         [30.0,8.0]| 25000|
|        [31.0,10.0]| 30000|
+-------------------+------+



In [30]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = df_final.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Feature', labelCol='Salary')
regressor=regressor.fit(train_data)

24/08/03 17:22:12 WARN Instrumentation: [f5589eee] regParam is zero, which might cause numerical instability and overfitting.


In [31]:
regressor.coefficients

DenseVector([-298.2456, 1771.9298])

In [32]:
regressor.intercept

21561.403508772833

In [33]:
pred_results = regressor.evaluate(test_data)

In [35]:
pred_results.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|17070.175438596598|
|         [30.0,8.0]| 25000|26789.473684210512|
+-------------------+------+------------------+

